Appears to be going very slow in google colab. Might just try on my computer.

Need to import necessary libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn # may need to do more specific things
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
from tqdm.auto import tqdm
from scipy import stats
from time import time


In [3]:
# Define evaluation metrics function. This is from Logistic Regression program.
def evaluate_model(y_true, y_pred, model_name, dataset_size, split_num, model_weights, model_biases):
    metrics = {
        'Model': model_name,
        'Dataset Size': dataset_size,
        'Split': split_num,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='weighted'),
        'Recall': recall_score(y_true, y_pred, average='weighted'),
        'F1 Score': f1_score(y_true, y_pred, average='weighted'),
        'Confusion Matrix': confusion_matrix(y_true, y_pred),
        'Weights': model_weights, # not strictly metrics but useful to save
        'Biases': model_biases,
    }

    # Class-specific metrics for binary classification
    if len(np.unique(y_true)) == 2:
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        metrics.update({
            'Sensitivity (TPR)': tp / (tp + fn),
            'Specificity (TNR)': tn / (tn + fp),
            'Precision (Class 1)': precision_score(y_true, y_pred, pos_label=1),
            'Recall (Class 1)': recall_score(y_true, y_pred, pos_label=1)
        })

    return metrics

In [3]:
# Testing a single split: Healhty vs. (diabetes or pre-diabetes)

# Setup
folderName = "pca_splits/pca_splits/"

train_file_name = folderName + f"train_size_100000_split_1.csv"
test_file_name = folderName + f"test_size_100000_split_1.csv"

train_df = pd.read_csv(train_file_name)
test_df = pd.read_csv(test_file_name)

# Make an array of results for storage
results = []


# Separate features and targets
pca_columns = [col for col in train_df.columns if col.startswith('PC')]
target_columns = ['Diabetes_0', 'Diabetes_1', 'Diabetes_2']

X_train = train_df[pca_columns]
X_test = test_df[pca_columns]

# Model 1: Non-diabetic (0) vs (Pre-diabetic or Diabetic) (1 or 2)
print("\nTraining Model 1: Non-diabetic vs (Pre-diabetic or Diabetic)")

# Create binary targets
y_train_model1 = np.where((train_df['Diabetes_1'] == 1) | (train_df['Diabetes_2'] == 1), 1, 0)
y_test_model1 = np.where((test_df['Diabetes_1'] == 1) | (test_df['Diabetes_2'] == 1), 1, 0)

# Using the following to help write code:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
# https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html#sphx-glr-auto-examples-neural-networks-plot-mlp-alpha-py

# Need to to a random search over hyperparameters to see what the best model is for testing.
# Hyperparameters I want to modify: alpha (L2 regularization): loguniform from E-6 to 1, hidden layer sizes: two layers each lognormal from 10 to 1000 neurons

num_test_times = 3 # number of alphas to test
max_iterations = 1000 # don't know what it needs to be, I'll increase to as large size as I need

# Testing to see how long it takes to train a single model
# 10 neurons, 10 neurons: 66 seconds
# 15, 15: 126 seconds
# 25, 25: 110 seconds
# 50, 50: 73 seconds. It might be due to early stopping. 
# May want to use my version of cross-validation from HW 3. 

"""
init_t = time()
curr_model = MLPClassifier(hidden_layer_sizes = (15,15),alpha = 10**-3, max_iter = max_iterations, early_stopping = True)
curr_model.fit(X_train,y_train_model1)
final_t = time()
print(final_t - init_t,"seconds have passed")
"""

# I was having trouble using randomsearchcv for hidden_layer_sizes
# so I'll use a combination of random search with alpha and manually
# implemented grid search for the sizes of the layers.

layersizes = [int(10**1.0), int(10**1.2), int(10**1.4)] # log uniform
best_layer_size = int(10**1.0)
best_f1 = 0
best_param = None
for layersize in tqdm(layersizes):
    print("Current layer size:",layersize)
    init_t = time()
    hyper_params = {
        "alpha": stats.loguniform.rvs(10**-6,1,size=num_test_times),
    }
    random_search = RandomizedSearchCV(MLPClassifier(hidden_layer_sizes = (layersize,layersize), max_iter = max_iterations, early_stopping = True),hyper_params,n_iter=num_test_times, scoring = 'f1',cv = 3)
    random_search.fit(X_train,y_train_model1)
    curr_params = random_search.best_params_
    curr_f1 = random_search.best_score_
    if curr_f1 > best_f1:
        best_f1 = curr_f1
        best_param = curr_params
        best_layer_size = layersize
    final_t = time()
    print("Took", final_t - init_t, "seconds")
print("Best layer size:", best_layer_size)
print("Best alpha:",best_param)
print("Best f1:",best_f1)

# Get best model and predictions
best_model1 = MLPClassifier(hidden_layer_sizes = (best_layer_size,best_layer_size), alpha=best_param['alpha'], max_iter = max_iterations, early_stopping = True)
best_model1.fit(X_train,y_train_model1)
y_pred_model1 = best_model1.predict(X_test)

# See how well model does
model1_metrics = evaluate_model(y_test_model1, y_pred_model1,"MLP_Model1", 10**5, 1, best_model1.coefs_, best_model1.intercepts_)
results.append(model1_metrics)

"""
# I'm going to use RandomizedSearchCV to do cross-validation
# Based on documentation and this tutorial: https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html#sphx-glr-auto-examples-applications-plot-face-recognition-py
hyper_params = {
    "alpha": stats.loguniform.rvs(10**-6,1,size=num_test_times),
    #"hidden_layer_sizes": [stats.loguniform.rvs(10,1000,size=num_test_times),stats.loguniform.rvs(10,1000,size=num_test_times)]
}
random_search = RandomizedSearchCV(MLPClassifier(max_iter = max_iterations, early_stopping = True),hyper_params,n_iter=num_test_times)
random_search.fit(X_train,y_train_model1)
"""

"""
# Keep track of f1s and params in single list. Each element is a tuple: f1, alpha, layer1, layer2
f1_list = []
# Generate random paramters ahead of time
alphas = stats.loguniform.rvs(10**-6,1,size=num_test_times)
layer1s = stats.loguniform.rvs(10,1000,size=num_test_times)
layer2s = stats.loguniform.rvs(10,1000,size=num_test_times)
for i in tqdm(range(num_test_times)):
  curr_alpha = alphas[i]
  curr_layer1 = layer1s[i]
  curr_layer2 = layer2s[i]
  # Test the model and get the f1 score
  curr_model = MLPClassifier(hidden_layer_sizes = (curr_layer1,curr_layer2),alpha = curr_alpha, max_iter = max_iterations, early_stopping = True)
  curr_model.fit(X_train,y_train_model1)
"""




Training Model 1: Non-diabetic vs (Pre-diabetic or Diabetic)


  0%|          | 0/3 [00:00<?, ?it/s]

Current layer size: 10
Took 414.52256441116333 seconds
Current layer size: 15
Took 486.2524480819702 seconds
Current layer size: 25
Took 1144.7593071460724 seconds
Best layer size: 25
Best alpha: {'alpha': 0.05093501458684488}
Best f1: 0.8796547630831797


'\n# Keep track of f1s and params in single list. Each element is a tuple: f1, alpha, layer1, layer2\nf1_list = []\n# Generate random paramters ahead of time\nalphas = stats.loguniform.rvs(10**-6,1,size=num_test_times)\nlayer1s = stats.loguniform.rvs(10,1000,size=num_test_times)\nlayer2s = stats.loguniform.rvs(10,1000,size=num_test_times)\nfor i in tqdm(range(num_test_times)):\n  curr_alpha = alphas[i]\n  curr_layer1 = layer1s[i]\n  curr_layer2 = layer2s[i]\n  # Test the model and get the f1 score\n  curr_model = MLPClassifier(hidden_layer_sizes = (curr_layer1,curr_layer2),alpha = curr_alpha, max_iter = max_iterations, early_stopping = True)\n  curr_model.fit(X_train,y_train_model1)\n'

In [7]:
# Testing a single split, but 2nd task

folderName = "pca_splits/pca_splits/"

train_file_name = folderName + f"train_size_100000_split_1.csv"
test_file_name = folderName + f"test_size_100000_split_1.csv"

train_df = pd.read_csv(train_file_name)
test_df = pd.read_csv(test_file_name)

# Make an array of results for storage
results = []


# Separate features and targets
pca_columns = [col for col in train_df.columns if col.startswith('PC')]
target_columns = ['Diabetes_0', 'Diabetes_1', 'Diabetes_2']

X_train = train_df[pca_columns]
X_test = test_df[pca_columns]

# Model 2: Pre-diabetic (1) vs Diabetic (1 or 2)
print("\nTraining Model 2: Pre-diabetic vs Diabetic")

# Filter samples that are either pre-diabetic or diabetic
train_mask = (train_df['Diabetes_1'] == 1) | (train_df['Diabetes_2'] == 1)
test_mask = (test_df['Diabetes_1'] == 1) | (test_df['Diabetes_2'] == 1)

X_train_model2 = X_train[train_mask]
X_test_model2 = X_test[test_mask]

# Create binary targets (1 for pre-diabetic, 0 for diabetic)
y_train_model2 = np.where(train_df.loc[train_mask, 'Diabetes_1'] == 1, 1, 0)
y_test_model2 = np.where(test_df.loc[test_mask, 'Diabetes_1'] == 1, 1, 0)

# Skip if not enough samples
if len(np.unique(y_train_model2)) < 2 or len(np.unique(y_test_model2)) < 2:
    print(f"Skipping Model 2 for size 100,000, split 1 - not enough samples")

# Using the following to help write code:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
# https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html#sphx-glr-auto-examples-neural-networks-plot-mlp-alpha-py

# Need to to a random search over hyperparameters to see what the best model is for testing.
# Hyperparameters I want to modify: alpha (L2 regularization): loguniform from E-6 to 1, hidden layer sizes: two layers each lognormal from 10 to 1000 neurons

num_test_times = 3 # number of alphas to test
max_iterations = 1000 # don't know what it needs to be, I'll increase to as large size as I need

# Testing to see how long it takes to train a single model
# 10 neurons, 10 neurons: 66 seconds
# 15, 15: 126 seconds
# 25, 25: 110 seconds
# 50, 50: 73 seconds. It might be due to early stopping. 
# May want to use my version of cross-validation from HW 3. 

"""
init_t = time()
curr_model = MLPClassifier(hidden_layer_sizes = (15,15),alpha = 10**-3, max_iter = max_iterations, early_stopping = True)
curr_model.fit(X_train,y_train_model1)
final_t = time()
print(final_t - init_t,"seconds have passed")
"""

# I was having trouble using randomsearchcv for hidden_layer_sizes
# so I'll use a combination of random search with alpha and manually
# implemented grid search for the sizes of the layers.

layersizes = [int(10**1.0), int(10**1.2), int(10**1.4)] # log uniform
best_layer_size = int(10**1.0)
best_f1 = 0
best_param = None
for layersize in tqdm(layersizes):
    print("Current layer size:",layersize)
    init_t = time()
    hyper_params = {
        "alpha": stats.loguniform.rvs(10**-6,1,size=num_test_times),
    }
    random_search = RandomizedSearchCV(MLPClassifier(hidden_layer_sizes = (layersize,layersize), max_iter = max_iterations, early_stopping = True),hyper_params,n_iter=num_test_times, scoring = 'f1',cv = 3)
    random_search.fit(X_train_model2,y_train_model2)
    curr_params = random_search.best_params_
    curr_f1 = random_search.best_score_
    if curr_f1 > best_f1:
        best_f1 = curr_f1
        best_param = curr_params
        best_layer_size = layersize
    final_t = time()
    print("Took", final_t - init_t, "seconds")
print("Best layer size:", best_layer_size)
print("Best alpha:",best_param)
print("Best f1:",best_f1)

# Get best model and predictions
best_model2 = MLPClassifier(hidden_layer_sizes = (best_layer_size,best_layer_size), alpha=best_param['alpha'], max_iter = max_iterations, early_stopping = True)
best_model2.fit(X_train_model2,y_train_model2)
y_pred_model2 = best_model2.predict(X_test)

# See how well model does
model2_metrics = evaluate_model(y_test_model2, y_pred_model2,"MLP_Model2", 10**5, 1, best_model2.coefs_, best_model2.intercepts_)
results.append(model2_metrics)

"""
# I'm going to use RandomizedSearchCV to do cross-validation
# Based on documentation and this tutorial: https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html#sphx-glr-auto-examples-applications-plot-face-recognition-py
hyper_params = {
    "alpha": stats.loguniform.rvs(10**-6,1,size=num_test_times),
    #"hidden_layer_sizes": [stats.loguniform.rvs(10,1000,size=num_test_times),stats.loguniform.rvs(10,1000,size=num_test_times)]
}
random_search = RandomizedSearchCV(MLPClassifier(max_iter = max_iterations, early_stopping = True),hyper_params,n_iter=num_test_times)
random_search.fit(X_train,y_train_model1)
"""

"""
# Keep track of f1s and params in single list. Each element is a tuple: f1, alpha, layer1, layer2
f1_list = []
# Generate random paramters ahead of time
alphas = stats.loguniform.rvs(10**-6,1,size=num_test_times)
layer1s = stats.loguniform.rvs(10,1000,size=num_test_times)
layer2s = stats.loguniform.rvs(10,1000,size=num_test_times)
for i in tqdm(range(num_test_times)):
  curr_alpha = alphas[i]
  curr_layer1 = layer1s[i]
  curr_layer2 = layer2s[i]
  # Test the model and get the f1 score
  curr_model = MLPClassifier(hidden_layer_sizes = (curr_layer1,curr_layer2),alpha = curr_alpha, max_iter = max_iterations, early_stopping = True)
  curr_model.fit(X_train,y_train_model1)
"""




Training Model 2: Pre-diabetic vs Diabetic


  0%|          | 0/3 [00:00<?, ?it/s]

Current layer size: 10
Took 752.4629020690918 seconds
Current layer size: 15
Took 1165.4849123954773 seconds
Current layer size: 25
Took 1577.3784255981445 seconds
Best layer size: 25
Best alpha: {'alpha': 7.721530999421516e-06}
Best f1: 0.7905554861099682


ValueError: Found input variables with inconsistent numbers of samples: [134546, 201819]

In [8]:
# Redoing this portion to avoid redoing whole thing
y_pred_model2 = best_model2.predict(X_test_model2)

# See how well model does
model2_metrics = evaluate_model(y_test_model2, y_pred_model2,"MLP_Model2", 10**5, 1, best_model2.coefs_, best_model2.intercepts_)
results.append(model2_metrics)

In [9]:
print(len(results))

1


In [10]:
print("Saving results")
# Convert results to DataFrame and save
results_df = pd.DataFrame(results)
results_file = "MLP_results_pre_vs_diabedes.csv"
results_df.to_csv(results_file, index=False)
print(f"\nSaved results to {results_file}")

# Print summary statistics
print("\nSummary Statistics:")
print(results_df.groupby(['Model', 'Dataset Size'])[['Accuracy', 'Precision', 'Recall', 'F1 Score']].mean())

Saving results

Saved results to MLP_results_pre_vs_diabedes.csv

Summary Statistics:
                         Accuracy  Precision   Recall  F1 Score
Model      Dataset Size                                        
MLP_Model2 100000         0.77789   0.779224  0.77789  0.777624
